# Strings and String Methods

*📚 Computer Programming II · 👨‍🏫 Dr. Arif Solmaz*


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⏱️ OTURUM ZAMANLAYICI — Bu hücreyi ilk çalıştırın!
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import time as _time, datetime as _dt, json as _json, os as _os
from IPython.display import display, HTML as _HTML

# ── Persistent tracker file (survives kernel restarts in same runtime) ──
_TRACKER_FILE = _os.path.join(
    '/content' if _os.path.isdir('/content') else '/tmp',
    '.cp2_session_Week_07.json'
)

def _load_tracker():
    """Load previous sessions from file."""
    try:
        with open(_TRACKER_FILE, 'r') as f:
            return _json.load(f)
    except (FileNotFoundError, _json.JSONDecodeError, ValueError):
        return {'sessions': [], 'total_heartbeats': 0}

def _save_tracker(data):
    """Persist tracker data to file."""
    try:
        with open(_TRACKER_FILE, 'w') as f:
            _json.dump(data, f)
    except OSError:
        pass

# ── Load any previous session data ──
_tracker = _load_tracker()
_prev_sessions = len(_tracker['sessions'])
_prev_hb = _tracker['total_heartbeats']

# ── Start new session ──
_SESSION_START = _time.time()
_SESSION_START_STR = _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
_HEARTBEATS = [_time.time()]
_CELLS_RUN = [0]
IDLE_THRESHOLD = 300   # 5 min

# Record this session start
_tracker['sessions'].append({
    'start': _SESSION_START_STR,
    'start_ts': _SESSION_START,
    'heartbeats': [_SESSION_START]
})
_save_tracker(_tracker)

def _heartbeat_hook(*args, **kwargs):
    """Record each cell execution — in-memory + file."""
    now = _time.time()
    _HEARTBEATS.append(now)
    _CELLS_RUN[0] += 1
    # Persist every 5 cells to avoid excessive disk IO
    if _CELLS_RUN[0] % 5 == 0:
        try:
            _t = _load_tracker()
            if _t['sessions']:
                _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
                _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
                _t['total_heartbeats'] = _prev_hb + len(_HEARTBEATS)
                _save_tracker(_t)
        except Exception:
            pass

def _calc_active_time(heartbeats=None):
    """Calculate active time from heartbeat list, capping idle gaps."""
    hb = heartbeats or _HEARTBEATS
    if len(hb) < 2: return 0
    active = 0
    for i in range(1, len(hb)):
        gap = hb[i] - hb[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _calc_total_active_time():
    """Calculate total across ALL sessions (multi-day support)."""
    total = 0
    try:
        _t = _load_tracker()
        for sess in _t['sessions'][:-1]:  # all previous sessions
            hb = sess.get('heartbeats', [])
            if len(hb) >= 2:
                for i in range(1, len(hb)):
                    gap = hb[i] - hb[i-1]
                    total += gap if gap <= IDLE_THRESHOLD else 30
    except Exception:
        pass
    # Add current session
    total += _calc_active_time()
    return int(total)

try:
    _ip = get_ipython()
    _ip.events.register('pre_run_cell', _heartbeat_hook)
except Exception: pass

# ── Display ──
_multi = ''
if _prev_sessions > 0:
    _multi = f'<br><span style="font-size:12px;opacity:.75">📂 {_prev_sessions} önceki oturum bulundu — çoklu oturum takibi aktif.</span>'
display(_HTML(f"""<div style='background:linear-gradient(135deg,#667eea,#764ba2);padding:14px 20px;border-radius:10px;color:white;font-family:system-ui;margin:4px 0'>
<b>⏱️ Oturum Başladı</b> — {_SESSION_START_STR}<br>
<span style='font-size:13px;opacity:.85'>Hücre aktiviteniz takip ediliyor. Bitince en alttaki Submit hücresini çalıştırın.</span>{_multi}</div>"""))
print(f'✅ Zamanlayıcı aktif. Idle eşiği: {IDLE_THRESHOLD//60} dk')
if _prev_sessions > 0:
    print(f'📂 Önceki oturumlar: {_prev_sessions} | Toplam heartbeat: {_prev_hb}')

## 🎯 Learning Objectives

- Understand string fundamentals and creation methods
- Master string indexing and slicing techniques
- Use case conversion methods effectively
- Search and find patterns within strings
- Modify and manipulate strings safely
- Split strings and join sequences
- Validate string content with built-in methods
- Format strings using f-strings and format()
- Apply regular expressions for pattern matching
- Process text data in engineering applications

---
## Part 1: String Fundamentals


A **string** is one of Python's most fundamental data types. It represents a sequence of characters enclosed in quotes. Strings are used extensively in programming for storing text data, processing files, handling user input, and communicating with databases and web services.

| Feature | Description |
| --- | --- |
| **Immutable** | Once created, strings cannot be changed |
| **Ordered** | Characters maintain their position (indexed from 0) |
| **Iterable** | Can loop through each character |
| **Unicode Support** | Python 3 supports all Unicode characters |
| **Hashable** | Can be used as dictionary keys |

### Creating Strings

**Figure 1.1: String Creation**

In [ ]:
# Single quotes - most common for short strings
single = 'Hello, World!'
print(f"Single quotes: {single}")

# Double quotes - useful when string contains apostrophes
double = "It's a beautiful day!"
print(f"Double quotes: {double}")

# Triple quotes - for multi-line strings
triple = '''This is a
multi-line
string'''
print(f"Triple quotes:\n{triple}")

print(f"\nType: {type(single)}")

### Escape Characters

| Escape | Meaning | Example |
| --- | --- | --- |
| `\n` | Newline | "Line1\nLine2" |
| `\t` | Tab | "Col1\tCol2" |
| `\\` | Backslash | "C:\\folder" |
| `\'` | Single quote | 'It\'s' |
| `\"` | Double quote | "Say \"Hi\"" |

**Figure 1.2: Escape Characters**

In [ ]:
# Escape characters
print("Line 1\nLine 2\nLine 3")  # Newlines
print()
print("Column1\tColumn2\tColumn3")  # Tabs
print()
print("Path: C:\\Users\\Student\\Documents")  # Backslash
print()
print("She said \"Hello!\"")  # Quotes

# Raw strings - no escape processing
raw = r"C:\Users\new\test"
print(f"\nRaw string: {raw}")

### String Concatenation and Repetition

**Figure 1.3: Concatenation**

In [ ]:
# Concatenation with +
first_name = "Ali"
last_name = "Yilmaz"
full_name = first_name + " " + last_name
print(f"Full name: {full_name}")

# Repetition with *
separator = "=" * 40
print(separator)
print("REPORT HEADER".center(40))
print(separator)

# String length
message = "Python Programming"
print(f"\nMessage: '{message}'")
print(f"Length: {len(message)} characters")

---
## Part 2: String Indexing and Slicing


Since strings are ordered sequences, each character has a position (index). Python uses zero-based indexing, meaning the first character is at index 0.

```
String:    P   y   t   h   o   n
Index:     0   1   2   3   4   5
Negative: -6  -5  -4  -3  -2  -1
```

### Basic Indexing

**Figure 2.1: String Indexing**

In [ ]:
# String indexing
text = "Python"

print(f"String: '{text}'")
print(f"Length: {len(text)}")

# Positive indexing (left to right)
print(f"\nPositive indexing:")
print(f"  text[0] = '{text[0]}'  (first)")
print(f"  text[1] = '{text[1]}'")
print(f"  text[5] = '{text[5]}'  (last)")

# Negative indexing (right to left)
print(f"\nNegative indexing:")
print(f"  text[-1] = '{text[-1]}'  (last)")
print(f"  text[-2] = '{text[-2]}'")
print(f"  text[-6] = '{text[-6]}'  (first)")

### String Slicing

**Figure 2.2: String Slicing**

In [ ]:
# String slicing [start:stop:step]
text = "Python Programming"

print(f"String: '{text}'")

# Basic slicing
print(f"\n[0:6] = '{text[0:6]}'")
print(f"[7:18] = '{text[7:18]}'")
print(f"[:6] = '{text[:6]}'  (from start)")
print(f"[7:] = '{text[7:]}'  (to end)")

# With step
print(f"\n[::2] = '{text[::2]}'  (every 2nd)")
print(f"[::-1] = '{text[::-1]}'  (reversed)")

> 💡 **Note:** **Strings are immutable!** You cannot change individual characters. `text[0] = 'p'` will raise a TypeError. Create a new string instead.

---
## Part 3: Case Conversion Methods


| Method | Description | Example |
| --- | --- | --- |
| `upper()` | All uppercase | "hello" → "HELLO" |
| `lower()` | All lowercase | "HELLO" → "hello" |
| `capitalize()` | First char uppercase | "hello" → "Hello" |
| `title()` | Each word capitalized | "hello world" → "Hello World" |
| `swapcase()` | Swap case | "HeLLo" → "hEllO" |

**Figure 3.1: Case Methods**

In [ ]:
text = "hello WORLD python"

print(f"Original: '{text}'")
print(f"upper():     '{text.upper()}'")
print(f"lower():     '{text.lower()}'")
print(f"capitalize(): '{text.capitalize()}'")
print(f"title():     '{text.title()}'")
print(f"swapcase():  '{text.swapcase()}'")

# Practical: Case-insensitive comparison
user_input = "YES"
if user_input.lower() == "yes":
    print("\nUser confirmed!")

---
## Part 4: Searching and Finding


| Method | Description | Not Found |
| --- | --- | --- |
| `find(sub)` | Find first occurrence | Returns -1 |
| `rfind(sub)` | Find last occurrence | Returns -1 |
| `index(sub)` | Find first occurrence | Raises ValueError |
| `count(sub)` | Count occurrences | Returns 0 |
| `startswith()` | Check prefix | Returns False |
| `endswith()` | Check suffix | Returns False |

**Figure 4.1: Search Methods**

In [ ]:
text = "Python is powerful. Python is popular."

print(f"Text: '{text}'")

# find() and rfind()
print(f"\nfind('Python'): {text.find('Python')}")
print(f"rfind('Python'): {text.rfind('Python')}")
print(f"find('Java'): {text.find('Java')}")

# count()
print(f"\ncount('Python'): {text.count('Python')}")
print(f"count('is'): {text.count('is')}")

# startswith() and endswith()
print(f"\nstartswith('Python'): {text.startswith('Python')}")
print(f"endswith('.'): {text.endswith('.')}")

### Practical: File Extension Checking

**Figure 4.2: File Validation**

In [ ]:
def validate_file(filename, allowed_extensions):
    """Check if file has allowed extension"""
    return filename.lower().endswith(tuple(allowed_extensions))

# Test files
files = ["report.pdf", "data.CSV", "image.PNG", "script.exe", "notes.txt"]
allowed = [".pdf", ".csv", ".txt"]

print("File Validation:")
print("-" * 40)
for f in files:
    status = "✓ Valid" if validate_file(f, allowed) else "✗ Invalid"
    print(f"  {f:20} {status}")

---
## Part 5: String Modification Methods


Since strings are immutable, these methods return **new strings** rather than modifying the original.

### replace() Method

**Figure 5.1: Replace Method**

In [ ]:
# replace(old, new, count)
text = "Hello, World! Hello, Python!"

# Replace all occurrences
print(f"Original: '{text}'")
print(f"Replace all: '{text.replace('Hello', 'Hi')}'")
print(f"Replace first: '{text.replace('Hello', 'Hi', 1)}'")

# Remove characters
print(f"Remove '!': '{text.replace('!', '')}'")

# Original unchanged
print(f"\nOriginal: '{text}'")

### strip(), lstrip(), rstrip()

**Figure 5.2: Stripping Whitespace**

In [ ]:
# Stripping whitespace
text = "   Hello, World!   "

print(f"Original: '{text}'")
print(f"strip():  '{text.strip()}'")
print(f"lstrip(): '{text.lstrip()}'")
print(f"rstrip(): '{text.rstrip()}'")

# Strip specific characters
data = "###sensor_001###"
print(f"\nOriginal: '{data}'")
print(f"Strip '#': '{data.strip('#')}'")

### Padding and Alignment

**Figure 5.3: Padding Methods**

In [ ]:
text = "Python"

# Center, left, right alignment
print(f"center(20): |{text.center(20)}|")
print(f"ljust(20):  |{text.ljust(20)}|")
print(f"rjust(20):  |{text.rjust(20)}|")

# Custom fill character
print(f"\ncenter(20, '-'): |{text.center(20, '-')}|")
print(f"rjust(20, '0'):  |{text.rjust(20, '0')}|")

# zfill for numbers
num = "42"
print(f"\nzfill(5): '{num.zfill(5)}'")

---
## Part 6: Split and Join


These methods convert between strings and lists, essential for parsing and formatting data.

### split() Method

**Figure 6.1: Splitting Strings**

In [ ]:
# split() - break string into list
text = "Hello World Python Programming"

# Default: split on whitespace
words = text.split()
print(f"split(): {words}")

# Split on specific delimiter
csv_data = "apple,banana,cherry,date"
fruits = csv_data.split(",")
print(f"split(','): {fruits}")

# Limit splits
print(f"split(',', 2): {csv_data.split(',', 2)}")

# splitlines() for multi-line
multi = "Line1\nLine2\nLine3"
lines = multi.splitlines()
print(f"\nsplitlines(): {lines}")

### join() Method

**Figure 6.2: Joining Strings**

In [ ]:
# join() - combine list into string
words = ["Hello", "World", "Python"]

# Join with space
sentence = " ".join(words)
print(f"' '.join(): '{sentence}'")

# Join with other delimiters
print(f"','.join(): '{','.join(words)}'")
print(f"' | '.join(): '{' | '.join(words)}'")
print(f"''.join(): '{''.join(words)}'")

# Practical: Create CSV line
data = ["Ali", "21", "Mechatronics", "3.65"]
csv_line = ",".join(data)
print(f"\nCSV line: {csv_line}")

### Practical: Parsing Sensor Data

**Figure 6.3: Parsing Data**

In [ ]:
# Parse sensor readings
raw_data = "TEMP_001:25.5:C,PRESS_001:101.3:kPa,HUMID_001:45:%"

# Split into individual readings
readings = raw_data.split(",")

print("Parsed Sensor Data:")
print("-" * 40)
for reading in readings:
    parts = reading.split(":")
    sensor_id = parts[0]
    value = float(parts[1])
    unit = parts[2]
    print(f"  {sensor_id}: {value} {unit}")

---
## Part 7: String Validation Methods


| Method | Returns True if... |
| --- | --- |
| `isalpha()` | All characters are letters |
| `isdigit()` | All characters are digits |
| `isalnum()` | All are letters or digits |
| `isspace()` | All are whitespace |
| `isupper()` | All letters are uppercase |
| `islower()` | All letters are lowercase |
| `isnumeric()` | All are numeric characters |

**Figure 7.1: Validation Methods**

In [ ]:
# String validation methods
test_strings = ["Hello", "123", "Hello123", "   ", "HELLO", "hello"]

for s in test_strings:
    print(f"'{s}':")
    print(f"  isalpha(): {s.isalpha()}")
    print(f"  isdigit(): {s.isdigit()}")
    print(f"  isalnum(): {s.isalnum()}")
    print()

### Practical: Input Validation

**Figure 7.2: Input Validation**

In [ ]:
def validate_student_id(student_id):
    """Validate student ID format"""
    # Must be exactly 10 digits
    if len(student_id) != 10:
        return False, "Must be 10 characters"
    if not student_id.isdigit():
        return False, "Must contain only digits"
    return True, "Valid"

# Test validation
test_ids = ["2024001234", "12345", "202400ABC1", "2024001234"]
print("Student ID Validation:")
print("-" * 40)
for sid in test_ids:
    valid, msg = validate_student_id(sid)
    status = "✓" if valid else "✗"
    print(f"  {status} '{sid}': {msg}")

---
## Part 8: String Formatting


Python offers multiple ways to format strings. F-strings (Python 3.6+) are the most modern and recommended approach.

### F-Strings (Formatted String Literals)

**Figure 8.1: F-Strings**

In [ ]:
# F-strings - most modern approach
name = "Ali"
age = 21
gpa = 3.6543

# Basic usage
print(f"Name: {name}, Age: {age}")

# Expressions in f-strings
print(f"Next year: {age + 1}")
print(f"Name uppercase: {name.upper()}")

# Number formatting
print(f"GPA (2 decimals): {gpa:.2f}")
print(f"Percentage: {gpa/4*100:.1f}%")

# Width and alignment
print(f"|{name:>10}|")  # Right align
print(f"|{name:<10}|")  # Left align
print(f"|{name:^10}|")  # Center

### Format Specifiers

**Figure 8.2: Format Specifiers**

In [ ]:
value = 1234567.89

# Different number formats
print(f"Default: {value}")
print(f"Comma: {value:,}")
print(f"2 decimals: {value:,.2f}")
print(f"Scientific: {value:.2e}")

# Integer formats
num = 255
print(f"\nDecimal: {num:d}")
print(f"Binary: {num:b}")
print(f"Hex: {num:x}")
print(f"Octal: {num:o}")

# Padding with zeros
print(f"\nPadded: {42:05d}")

### Engineering: Formatted Reports

**Figure 8.3: Formatted Reports**

In [ ]:
# Create formatted sensor report
sensors = [
    ("TEMP_001", 25.5, "°C"),
    ("PRESS_001", 101.325, "kPa"),
    ("HUMID_001", 45.8, "%"),
    ("VOLT_001", 3.3, "V")
]

print("=" * 40)
print(f"{'SENSOR REPORT':^40}")
print("=" * 40)
print(f"{'ID':<12} {'VALUE':>10} {'UNIT':<6}")
print("-" * 40)
for sensor_id, value, unit in sensors:
    print(f"{sensor_id:<12} {value:>10.2f} {unit:<6}")
print("=" * 40)

---
## Part 9: Introduction to Regular Expressions


Regular expressions (regex) provide powerful pattern matching capabilities. Python's `re` module implements regex support.

| Pattern | Meaning |
| --- | --- |
| `\d` | Any digit (0-9) |
| `\w` | Any word character (a-z, A-Z, 0-9, _) |
| `\s` | Any whitespace |
| `.` | Any character except newline |
| `*` | Zero or more of previous |
| `+` | One or more of previous |
| `?` | Zero or one of previous |
| `{n}` | Exactly n occurrences |

**Figure 9.1: Basic Regex**

In [ ]:
import re

text = "Contact: ali@email.com or 0555-123-4567"

# Find email pattern
email_pattern = r'\w+@\w+\.\w+'
email = re.search(email_pattern, text)
if email:
    print(f"Found email: {email.group()}")

# Find phone pattern
phone_pattern = r'\d{4}-\d{3}-\d{4}'
phone = re.search(phone_pattern, text)
if phone:
    print(f"Found phone: {phone.group()}")

# Find all digits
digits = re.findall(r'\d+', text)
print(f"All digit groups: {digits}")

### Practical: Data Extraction

**Figure 9.2: Data Extraction**

In [ ]:
import re

# Extract sensor readings from log
log = """
[10:30:15] TEMP_001: 25.5°C
[10:30:16] PRESS_001: 101.3kPa
[10:30:17] HUMID_001: 45%
"""

# Pattern: sensor_id: value unit
pattern = r'(\w+_\d+): ([\d.]+)(\S+)'
matches = re.findall(pattern, log)

print("Extracted Readings:")
print("-" * 30)
for sensor_id, value, unit in matches:
    print(f"  {sensor_id}: {value} {unit}")

---
## Part 10: Engineering Applications


#### Log File Parser

**Figure 10.1: Log Parser**

In [ ]:
def parse_log_entry(line):
    """Parse a log entry into components"""
    # Format: [TIMESTAMP] LEVEL: MESSAGE
    parts = line.split("] ", 1)
    timestamp = parts[0][1:]  # Remove leading [
    
    level_msg = parts[1].split(": ", 1)
    level = level_msg[0]
    message = level_msg[1] if len(level_msg) > 1 else ""
    
    return {"timestamp": timestamp, "level": level, "message": message}

# Sample log entries
logs = [
    "[2024-01-15 10:30:45] INFO: System started",
    "[2024-01-15 10:31:02] WARNING: Temperature high",
    "[2024-01-15 10:32:15] ERROR: Sensor offline"
]

print("Parsed Log Entries:")
print("-" * 50)
for log in logs:
    parsed = parse_log_entry(log)
    print(f"  Time: {parsed['timestamp']}")
    print(f"  Level: {parsed['level']}")
    print(f"  Message: {parsed['message']}")
    print()

#### G-Code Generator

**Figure 10.2: G-Code Generator**

In [ ]:
def generate_gcode_line(x, y, z, feed_rate=100):
    """Generate G-code movement command"""
    return f"G1 X{x:.3f} Y{y:.3f} Z{z:.3f} F{feed_rate}"

# Generate simple path
points = [
    (0, 0, 5),
    (10, 0, 5),
    (10, 10, 5),
    (0, 10, 5),
    (0, 0, 5)
]

print("; Square path G-code")
print("G21 ; mm mode")
print("G90 ; absolute positioning")
print()
for x, y, z in points:
    print(generate_gcode_line(x, y, z))

---
## ❌ Common Mistakes to Avoid


These are the most frequent errors students make with strings. Study them before the exercises!

**Strings are immutable — methods return NEW strings**

`name = "hello"; name.upper()` does NOT change `name`! You must reassign: `name = name.upper()`. All string methods return a new string.

**Forgetting that `.split()` returns a list**

`"a,b,c".split(",")` returns `["a", "b", "c"]` — a list, not a string. You need to index or loop through the result.

**`.find()` returns -1, not an error**

`"hello".find("x")` returns `-1` (not found), which is truthy! Always check `if result != -1:`. Use `.index()` if you want a `ValueError` instead.

**Mixing up `.replace()` behavior**

`"aaa".replace("a", "b")` replaces ALL occurrences → `"bbb"`. For just the first: `"aaa".replace("a", "b", 1)` → `"baa"`.

**Escaping issues in strings**

`"path\new\folder"` interprets `\n` as newline! Use raw strings `r"path\new\folder"` or double backslashes `"path\\new\\folder"`.

---
# 📝 Exercises


### Exercise 1: String Length  (Easy)

Find the length of the string "Mechatronics Engineering".

**Expected Output:**
```
Length: 24
```

<details>
<summary>💡 Hints</summary>

- Use `len()` function
- Syntax: `len(string)`
- Assign to variable: `length = len(text)`
</details>

In [ ]:
# ✏️ [EX1]
text = "Mechatronics Engineering"
# Find the length

print(f"Length: {length}")

### Exercise 2: First and Last Character  (Easy)

Print the first and last character of "Python".

**Expected Output:**
```
First: P, Last: n
```

<details>
<summary>💡 Hints</summary>

- First character: `text[0]`
- Last character: `text[-1]`
- Negative index counts from end
</details>

In [ ]:
# ✏️ [EX2]
text = "Python"
# Get first and last characters

print(f"First: {first}, Last: {last}")

### Exercise 3: Slice Middle  (Easy)

Extract "Programming" from "Python Programming".

**Expected Output:**
```
Programming
```

<details>
<summary>💡 Hints</summary>

- Use slicing: `text[start:end]`
- "Programming" starts at index 7
- Or use: `text[7:]` to get from 7 to end
</details>

In [ ]:
# ✏️ [EX3]
text = "Python Programming"
# Extract "Programming" using slicing
result = 
print(result)

### Exercise 4: Reverse String  (Easy)

Reverse the string "Hello" using slicing.

**Expected Output:**
```
olleH
```

<details>
<summary>💡 Hints</summary>

- Use slice with step -1: `[::-1]`
- Syntax: `text[::-1]`
- This creates a reversed copy
</details>

In [ ]:
# ✏️ [EX4]
text = "Hello"
# Reverse using slicing
reversed_text = 
print(reversed_text)

### Exercise 5: Convert to Uppercase  (Easy)

Convert "hello world" to uppercase.

**Expected Output:**
```
HELLO WORLD
```

<details>
<summary>💡 Hints</summary>

- Use `.upper()` method
- Syntax: `text.upper()`
- Returns new string, doesn't modify original
</details>

In [ ]:
# ✏️ [EX5]
text = "hello world"
# Convert to uppercase
result = 
print(result)

### Exercise 6: Count Occurrences  (Easy)

Count how many times "the" appears in the text.

**Expected Output:**
```
Count: 3
```

<details>
<summary>💡 Hints</summary>

- Use `.count()` method
- Syntax: `text.count("substring")`
- Returns number of non-overlapping occurrences
</details>

In [ ]:
# ✏️ [EX6]
text = "the quick brown fox jumps over the lazy dog the end"
# Count "the"
count = 
print(f"Count: {count}")

### Exercise 7: Replace Words  (Medium)

Replace "Python" with "Java" in the text.

**Expected Output:**
```
I love Java programming
```

<details>
<summary>💡 Hints</summary>

- Use `.replace()` method
- Syntax: `text.replace("old", "new")`
- Returns new string with replacements
</details>

In [ ]:
# ✏️ [EX7]
text = "I love Python programming"
# Replace Python with Java
result = 
print(result)

### Exercise 8: Split and Join  (Medium)

Split the CSV data by comma and join with " | ".

**Expected Output:**
```
apple | banana | cherry
```

<details>
<summary>💡 Hints</summary>

- Split: `text.split(",")` returns list
- Join: `" | ".join(list)`
- Chain: `" | ".join(text.split(","))`
</details>

In [ ]:
# ✏️ [EX8]
csv_data = "apple,banana,cherry"
# Split by comma, then join with " | "

print(result)

### Exercise 9: Check File Extension  (Medium)

Check if the filename ends with ".csv" or ".txt".

**Expected Output:**
```
Valid data file: True
```

<details>
<summary>💡 Hints</summary>

- Use `.endswith()` method
- Can pass tuple: `endswith((".csv", ".txt"))`
- Returns True if ends with any of them
</details>

In [ ]:
# ✏️ [EX9]
filename = "sensor_data.csv"
# Check if ends with .csv or .txt
is_valid = 
print(f"Valid data file: {is_valid}")

### Exercise 10: Strip and Title  (Medium)

Clean the input by stripping whitespace and converting to title case.

**Expected Output:**
```
Hello World Python
```

<details>
<summary>💡 Hints</summary>

- Strip whitespace: `.strip()`
- Title case: `.title()`
- Chain: `text.strip().title()`
</details>

In [ ]:
# ✏️ [EX10]
user_input = "   hello world python   "
# Strip whitespace and convert to title case
cleaned = 
print(cleaned)

### Exercise 11: F-String Formatting  (Medium)

Format the number with 2 decimal places and comma separator.

**Expected Output:**
```
Value: 1,234,567.89
```

<details>
<summary>💡 Hints</summary>

- Format spec: `{value:,.2f}`
- Comma for thousands separator
- .2f for 2 decimal places
</details>

In [ ]:
# ✏️ [EX11]
value = 1234567.89
# Format with commas and 2 decimals
print(f"Value: {}")

### Exercise 12: Validate Digit String  (Medium)

Check if the string contains only digits.

**Expected Output:**
```
Is all digits: True
```

<details>
<summary>💡 Hints</summary>

- Use `.isdigit()` method
- Returns True if all characters are digits
- Syntax: `text.isdigit()`
</details>

In [ ]:
# ✏️ [EX12]
student_id = "2024001234"
# Check if all characters are digits
is_valid = 
print(f"Is all digits: {is_valid}")

### Exercise 13: Word Counter  (Challenge)

Count the number of words in the sentence.

**Expected Output:**
```
Word count: 6
```

<details>
<summary>💡 Hints</summary>

- Use `split()` to break into words
- Use `len()` to count the list
</details>

In [ ]:
# ✏️ [EX13]
sentence = "Python is a powerful programming language"
# Count the words
word_count = 
print(f"Word count: {word_count}")

### Exercise 14: Palindrome Check  (Challenge)

Check if "radar" is a palindrome (reads same forwards and backwards).

**Expected Output:**
```
Is palindrome: True
```

<details>
<summary>💡 Hints</summary>

- Compare string with its reverse
- Use `[::-1]` to reverse
</details>

In [ ]:
# ✏️ [EX14]
word = "radar"
# Check if palindrome
is_palindrome = 
print(f"Is palindrome: {is_palindrome}")

### Exercise 15: Email Validator  (Challenge)

Validate that email contains "@" and ends with ".com" or ".edu".

**Expected Output:**
```
Valid email: True
```

<details>
<summary>💡 Hints</summary>

- Use `"@" in email` to check for @
- Use `endswith()` with a tuple
</details>

In [ ]:
# ✏️ [EX15]
email = "student@university.edu"
# Validate email format
is_valid = 
print(f"Valid email: {is_valid}")

### Exercise 🌉: Bridge Exercise: Sneak Peek at Week 8  (Preview)

**Next week: File I/O!** You've built a perfect sensor data parser using string methods — but when the program ends, all your data disappears! What if you could save it to a file and reload it later?

**Expected Output:**
```
Parsed sensor data:
  temp_01: 24.5 (OK)
  humidity_01: 65.2 (WARNING)

All this data is lost when the program ends! 😫
Can't share with colleagues either...
Next week: File I/O saves data permanently!
```

<details>
<summary>💡 Hints</summary>

- We can parse strings beautifully now, but the results only live in memory
- When the script finishes, everything vanishes
- Next week: `with open("data.txt", "w") as f:` — save to disk permanently
</details>

In [ ]:
# ✏️ [EXBridge]
# Bridge Exercise: Data That Disappears!
# Using string methods to parse sensor logs

raw_logs = [
    "  TEMP: 24.5C | STATUS: OK  ",
    "  HUMIDITY: 65.2% | STATUS: WARNING  ",
]

parsed_data = []
for log in raw_logs:
    parts = log.strip().split("|")
    sensor_info = parts[0].strip().split(":")
    status = parts[1].strip().split(":")[1].strip()
    sensor_name = sensor_info[0].strip().lower().replace(" ", "_")
    value = sensor_info[1].strip().rstrip("C%")
    parsed_data.append({"name": sensor_name, "value": value, "status": status})

print("Parsed sensor data:")
for item in parsed_data:
    print(f"  {item['name']}: {item['value']} ({item['status']})")

# But when the program ends...
print("\nAll this data is lost when the program ends! 😫")
print("Can't share with colleagues either...")
print("Next week: File I/O saves data permanently!")

---
# 📮 Submit Your Work

**When you're done with all exercises:**
1. **Run all exercise cells** (make sure each one executed)
2. Fill in your info in the cell below and run it
3. Run the next cell to submit


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 1: Fill in your info below, then run this cell
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

STUDENT_ID    = ""     # e.g. "2024001234"
STUDENT_NAME  = ""     # e.g. "Ahmet Yılmaz"
STUDENT_EMAIL = ""     # e.g. "ahmet.yilmaz@istun.edu.tr"
CLASS_CODE    = ""     # code given in class

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Don't change anything below this line
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import re as _re

_errors = []
if not _re.match(r"^\d{6,12}$", STUDENT_ID):
    _errors.append("❌ Student ID must be 6-12 digits")
if len(STUDENT_NAME.strip().split()) < 2:
    _errors.append("❌ Enter first and last name")
if not STUDENT_EMAIL.strip().lower().endswith("@istun.edu.tr") or len(STUDENT_EMAIL.strip()) < 16:
    _errors.append("❌ Use your @istun.edu.tr email")
if len(CLASS_CODE.strip()) < 4:
    _errors.append("❌ Invalid class code")

if _errors:
    for _e in _errors:
        print(_e)
    print("\n⚠️  Fix the errors above and run this cell again.")
else:
    print(f"✅ Info OK — {STUDENT_NAME} ({STUDENT_ID})")
    print(f"   {STUDENT_EMAIL}")
    print(f"\n👉 Now run the NEXT cell to submit.")

In [ ]:
_ORIGINAL_HASHES = {
    "ex1": "4349516eae4b3879e8bc25e8cd4ef2f3",
    "ex2": "6808fd94eb73a27bf4ee92c807c04f50",
    "ex3": "afeeaac8d89ed55eb523d8be2ea517d8",
    "ex4": "ec846b92a4d2f0b21b6dd6138628e1ae",
    "ex5": "e2180910df1d7a9ee55d0b3eb3bb7fe6",
    "ex6": "afad45db528e5b7b8cb61854a5b9df62",
    "ex7": "49e4644c2874d77541180dcc0cd3a12e",
    "ex8": "66e8e7e3328d08fba42d10df3424aace",
    "ex9": "c82eda28a515fecf32a0554524b84f2f",
    "ex10": "5126aa7aebb992fcb35b3430da34ef8e",
    "ex11": "4124f55ccc9466ed537a2703dc5d74bb",
    "ex12": "043ceeb4826eb9712a3c4d382124afd7",
    "ex13": "1566bf4e8cb703d01b0a276d1e1183c7",
    "ex14": "4eaef2011349b4ace014069f05c2251a",
    "ex15": "b25141ffeee5a6ac0ed43c05671c1fee"
}

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 2: Run this cell to submit
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⚠️  Make sure you RAN all exercise cells first!

import json, re, os, urllib.request, hashlib
import time as _time, datetime as _dt

WEEK = "Week_07"
URL  = "https://script.google.com/macros/s/AKfycbyf1D3HGSAX4MoIhNlAuWlGrFyyvbM5MIv7ZsLxrVDlATUihrRGEAaibvIZYlCfd8Me/exec"

# ══════════════════════════════════════════════════════════
# SELF-HEALING TIMER — works even if Cell 1 was skipped
# ══════════════════════════════════════════════════════════
_TRACKER_FILE = os.path.join(
    '/content' if os.path.isdir('/content') else '/tmp',
    f'.cp2_session_{WEEK}.json'
)
IDLE_THRESHOLD = 300

def _sh_calc_active(heartbeats):
    """Calculate active seconds from a heartbeat list."""
    if len(heartbeats) < 2: return 0
    active = 0
    for i in range(1, len(heartbeats)):
        gap = heartbeats[i] - heartbeats[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _collect_timing():
    """
    Collect timing data from all available sources:
      1. In-memory variables (timer cell ran in this kernel)
      2. Persistent tracker file (previous sessions / kernel restarts)
      3. Minimal fallback (nothing available)
    """
    _info = {
        'active_time': 0, 'wall_time': 0, 'cells_run': 0,
        'heartbeat_count': 0, 'session_count': 0,
        'session_start': '', 'session_end': _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'source': 'none'
    }

    # ── Source 1: In-memory (timer cell ran this kernel) ──
    _has_memory = False
    try:
        _s = _SESSION_START        # noqa: F821
        _hb = list(_HEARTBEATS)    # noqa: F821
        _cr = _CELLS_RUN[0]        # noqa: F821
        _has_memory = True
        _info['active_time'] = _sh_calc_active(_hb)
        _info['wall_time'] = int(_time.time() - _s)
        _info['cells_run'] = _cr
        _info['heartbeat_count'] = len(_hb)
        _info['session_start'] = _SESSION_START_STR  # noqa: F821
        _info['source'] = 'memory'
    except NameError:
        pass

    # ── Source 2: Persistent file (adds previous sessions) ──
    try:
        with open(_TRACKER_FILE, 'r') as f:
            _t = json.load(f)
        sessions = _t.get('sessions', [])
        _info['session_count'] = len(sessions)

        if not _has_memory and sessions:
            # Timer cell was NOT run — reconstruct from file
            last = sessions[-1]
            hb = last.get('heartbeats', [])
            _info['active_time'] = _sh_calc_active(hb)
            _info['wall_time'] = int(hb[-1] - hb[0]) if len(hb) >= 2 else 0
            _info['cells_run'] = last.get('cells_run', len(hb))
            _info['heartbeat_count'] = len(hb)
            _info['session_start'] = last.get('start', '')
            _info['source'] = 'file-current'

        # Add previous sessions' active time
        if len(sessions) > 1:
            prev_active = 0
            prev_cells = 0
            prev_hb = 0
            cutoff = -1 if _has_memory else -1  # exclude current session
            for sess in sessions[:cutoff]:
                hb = sess.get('heartbeats', [])
                prev_active += _sh_calc_active(hb)
                prev_cells += sess.get('cells_run', len(hb))
                prev_hb += len(hb)
            _info['prev_active_time'] = prev_active
            _info['prev_cells_run'] = prev_cells
            _info['prev_heartbeats'] = prev_hb
            _info['total_active_time'] = _info['active_time'] + prev_active
            _info['total_cells_run'] = _info['cells_run'] + prev_cells
            if _info['source'] == 'none':
                _info['source'] = 'file-prev'

    except (FileNotFoundError, json.JSONDecodeError, KeyError, ValueError):
        pass

    # Ensure totals exist
    _info.setdefault('total_active_time', _info['active_time'])
    _info.setdefault('total_cells_run', _info['cells_run'])
    return _info

# ── Persist final heartbeat snapshot before collecting ──
try:
    _t = json.load(open(_TRACKER_FILE))
    if _t['sessions']:
        _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
        _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
        json.dump(_t, open(_TRACKER_FILE, 'w'))
except Exception:
    pass

_timing = _collect_timing()

# ── Display timing ──
if _timing['source'] == 'none':
    print("⚠️  Zamanlayıcı verisi bulunamadı — ilk hücreyi çalıştırmayı unutmayın.")
    print("    Gönderim yine de yapılacak (süre: 0 olarak kaydedilir).")
else:
    _a = _timing['active_time']
    _w = _timing['wall_time']
    print(f"⏱️  Bu oturum — Aktif: {_a//60}m {_a%60}s  |  Duvar: {_w//60}m {_w%60}s")
    print(f"🔢  Hücreler: {_timing['cells_run']}  |  Heartbeat: {_timing['heartbeat_count']}")
    if _timing['session_count'] > 1:
        _ta = _timing['total_active_time']
        print(f"📂  Toplam {_timing['session_count']} oturum — Toplam aktif: {_ta//60}m {_ta%60}s  |  Toplam hücre: {_timing['total_cells_run']}")
    if _timing['total_active_time'] < 120:
        print("⚠️  Toplam aktif süre < 2 dk — çalıştırdığınız hücre sayısı düşük olabilir.")

_time_on_page = _timing['total_active_time']

# ══════════════════════════════════════════════════════════
# STUDENT INFO CHECK
# ══════════════════════════════════════════════════════════
try:
    _sid = STUDENT_ID.strip()
    _sname = STUDENT_NAME.strip()
    _semail = STUDENT_EMAIL.strip().lower()
    _scode = CLASS_CODE.strip().upper()
except NameError:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın (öğrenci bilgileri).")

if not _sid or not _sname or not _semail or not _scode:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın — bazı alanlar boş.")

# ══════════════════════════════════════════════════════════
# EXERCISE ANSWER COLLECTION (improved with retry)
# ══════════════════════════════════════════════════════════
_EX_PATTERN = re.compile(r'#\s*✏️\s*\[EX(\d+)\]')

def _extract_exercise(src):
    """Try to extract exercise ID and code from a source string."""
    m = _EX_PATTERN.match(src)
    if not m: return None, None
    ex_id = 'ex' + m.group(1)
    clean = '\n'.join(src.split('\n')[1:]).strip()
    return ex_id, clean

_answers = {}

# ── Method 1: In[] list (most reliable for Run-All) ──
try:
    for _src in In:
        if not _src: continue
        _eid, _code = _extract_exercise(_src)
        if _eid:
            _answers[_eid] = {
                'code': _code,
                'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
            }
except NameError:
    pass

# ── Method 2: IPython history_manager (flush first!) ──
if not _answers:
    try:
        _ipy = get_ipython()
        # Force flush so Run-All history is fully written
        try: _ipy.history_manager.db.commit()
        except Exception: pass
        _time.sleep(0.5)  # small delay for DB sync
        for _sess, _line, _src in _ipy.history_manager.get_range(output=False):
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
                }
    except Exception:
        pass

# ── Method 3: Read notebook file (VS Code / local Jupyter) ──
if not _answers:
    _nb_path = None
    try:
        _nb_path = __vsc_ipynb_file__
    except NameError:
        _candidates = [f for f in os.listdir('.') if f.endswith('.ipynb') and WEEK in f]
        if len(_candidates) == 1:
            _nb_path = _candidates[0]
    if _nb_path and os.path.exists(str(_nb_path)):
        with open(str(_nb_path), 'r', encoding='utf-8') as _f:
            _nb = json.load(_f)
        for _cell in _nb['cells']:
            if _cell['cell_type'] != 'code': continue
            _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
                }

# ── Method 4: Colab notebook file at /content/ ──
if not _answers:
    try:
        _colab_candidates = [f for f in os.listdir('/content') if f.endswith('.ipynb') and WEEK in f]
        if _colab_candidates:
            _nb_path = os.path.join('/content', _colab_candidates[0])
            with open(_nb_path, 'r', encoding='utf-8') as _f:
                _nb = json.load(_f)
            for _cell in _nb['cells']:
                if _cell['cell_type'] != 'code': continue
                _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
                _eid, _code = _extract_exercise(_src)
                if _eid:
                    _answers[_eid] = {
                        'code': _code,
                        'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
                    }
    except Exception:
        pass

print(f"📝 Found {len(_answers)} exercise(s): {', '.join(sorted(_answers.keys(), key=lambda x: int(x[2:]) if x[2:].isdigit() else 999))}")

if not _answers:
    print("\n⚠️  Hiçbir exercise yanıtı bulunamadı!")
    print("Submit'ten ÖNCE tüm exercise hücrelerini çalıştırdığınızdan emin olun.")
    print("\n💡 İpucu: 'Run All' yerine exercise hücrelerini tek tek çalıştırıp")
    print("   ardından bu hücreyi çalıştırmayı deneyin.")
    raise SystemExit()

# ══════════════════════════════════════════════════════════
# SEND SUBMISSION
# ══════════════════════════════════════════════════════════
_data = json.dumps({
    'week': WEEK,
    'studentId': _sid,
    'studentName': _sname,
    'studentEmail': _semail,
    'classCode': _scode,
    'source': 'cp2-notebook',
    'timeOnPage': _time_on_page,
    'wallTime': _timing['wall_time'],
    'cellsRun': _timing['total_cells_run'],
    'sessionCount': _timing['session_count'],
    'sessionStart': _timing['session_start'],
    'sessionEnd': _timing['session_end'],
    'timerSource': _timing['source'],
    'answers': _answers
}).encode('utf-8')

print("📡 Gönderiliyor...")

try:
    _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
    _resp = urllib.request.urlopen(_req, timeout=30)
    _result = json.loads(_resp.read().decode())
    if _result.get('success'):
        print(f"\n✅ {_result['message']}")
        print('📧 Onay için e-postanızı kontrol edin.')
    else:
        print(f"\n❌ {_result.get('message', 'Gönderim başarısız')}")
except Exception as _e:
    try:
        _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
        urllib.request.urlopen(_req, timeout=10)
    except:
        pass
    print(f"\n⚠️  İstek gönderildi — onay için e-postanızı kontrol edin.")
    print(f"(E-posta gelmezse tekrar deneyin veya hocaya başvurun)")
